This code exists to translate output tiles from datacube-stats back to where they need to go.

In [3]:
import os
import subprocess

In [4]:
#change to the directory containing our list of tiles
os.chdir('/g/data/r78/datacube_stats/TCW_stats/Surat/Surat_2yr/')

#get the list of tiles from the tile list file
tile_list=[]
with open('tile_list.txt', 'r') as tile_list_file:
    for line in tile_list_file:
        tile_list.append(line.strip('\n'))   
    #print(line, sep= '', end ='')
    #tile_list = tile_list_file.read()

print(tile_list)
tile_list_updated=[]
#replace spaces with underscores
for tile in tile_list:
    tile2= tile.replace(' ','_')
    tile_list_updated.append(tile2)
print(tile_list_updated)

['15 -27', '15 -28', '15 -29', '15 -30', '15 -31', '15 -32', '16 -27', '16 -28', '16 -29', '16 -30', '16 -31', '16 -32', '16 -33', '17 -27', '17 -28', '17 -29', '17 -30', '17 -31', '17 -32', '17 -33', '18 -29', '18 -30', '18 -31', '18 -32', '18 -33', '19 -31', '19 -32', '19 -33']
['15_-27', '15_-28', '15_-29', '15_-30', '15_-31', '15_-32', '16_-27', '16_-28', '16_-29', '16_-30', '16_-31', '16_-32', '16_-33', '17_-27', '17_-28', '17_-29', '17_-30', '17_-31', '17_-32', '17_-33', '18_-29', '18_-30', '18_-31', '18_-32', '18_-33', '19_-31', '19_-32', '19_-33']


In [5]:
os.listdir()

['Surat2yr.o8357541',
 'surat_2010_2018_2yr.yaml',
 'tile_list.txt',
 'task.pickle',
 'outputs_corrected',
 'Surat2yr.e8356508',
 'surat_1018.sh',
 'Surat2yr.o8356508',
 'Surat2yr.e8357541',
 'outputs']

In [6]:
os.chdir('/g/data/r78/datacube_stats/TCW_stats/Surat/Surat_2yr/outputs_corrected//')
#get the lower left and upper right coordinates from the output files where our output files worked

In [7]:
!pwd

/g/data1a/r78/datacube_stats/TCW_stats/Surat/Surat_2yr/outputs_corrected


In [8]:
tile_list_updated

['15_-27',
 '15_-28',
 '15_-29',
 '15_-30',
 '15_-31',
 '15_-32',
 '16_-27',
 '16_-28',
 '16_-29',
 '16_-30',
 '16_-31',
 '16_-32',
 '16_-33',
 '17_-27',
 '17_-28',
 '17_-29',
 '17_-30',
 '17_-31',
 '17_-32',
 '17_-33',
 '18_-29',
 '18_-30',
 '18_-31',
 '18_-32',
 '18_-33',
 '19_-31',
 '19_-32',
 '19_-33']

In [9]:
os.getcwd()

'/g/data1a/r78/datacube_stats/TCW_stats/Surat/Surat_2yr/outputs_corrected'

In [10]:
files_to_get_spatial_from = []
for tile in tile_list_updated:
    for files in os.listdir(): 
        if tile and '20140101__20160101.yaml' in files:
            files_to_get_spatial_from.append(files)
spatial_files =set(files_to_get_spatial_from)
spatial_files=list(spatial_files)
#check that we have the same amount of files as tiles
len(spatial_files)==len(tile_list_updated)
spatial_files.sort()

for i in range(0, len(spatial_files)):
    print(tile_list_updated[i], spatial_files[i])

15_-27 Surat_2yr_15_-27_20140101__20160101.yaml
15_-28 Surat_2yr_15_-28_20140101__20160101.yaml
15_-29 Surat_2yr_15_-29_20140101__20160101.yaml
15_-30 Surat_2yr_15_-30_20140101__20160101.yaml
15_-31 Surat_2yr_15_-31_20140101__20160101.yaml
15_-32 Surat_2yr_15_-32_20140101__20160101.yaml
16_-27 Surat_2yr_16_-27_20140101__20160101.yaml
16_-28 Surat_2yr_16_-28_20140101__20160101.yaml
16_-29 Surat_2yr_16_-29_20140101__20160101.yaml
16_-30 Surat_2yr_16_-30_20140101__20160101.yaml
16_-31 Surat_2yr_16_-31_20140101__20160101.yaml
16_-32 Surat_2yr_16_-32_20140101__20160101.yaml
16_-33 Surat_2yr_16_-33_20140101__20160101.yaml
17_-27 Surat_2yr_17_-27_20140101__20160101.yaml
17_-28 Surat_2yr_17_-28_20140101__20160101.yaml
17_-29 Surat_2yr_17_-29_20140101__20160101.yaml
17_-30 Surat_2yr_17_-30_20140101__20160101.yaml
17_-31 Surat_2yr_17_-31_20140101__20160101.yaml
17_-32 Surat_2yr_17_-32_20140101__20160101.yaml
17_-33 Surat_2yr_17_-33_20140101__20160101.yaml
18_-29 Surat_2yr_18_-29_20140101__201601

In [11]:
len(tile_list_updated)

28

In [12]:
years= ['20100101__20120101','20120101__20140101','20140101__20160101','20160101__20180101']

In [18]:
for year in years:
    for j in range(0,len(spatial_files)):
        with open(spatial_files[j], 'r') as spatialfileopen:       
        #print(spatialfileopen.readlines())
            ulx=uly=lrx=lry=None
            for line in spatialfileopen:               
                if 'lr: {x:' in line and lrx is None:
                    lr = line
                    lrx=  lr[14:23]
                    lry= lr[27:38]
                            #get uly and uly if we haven't found it yet. Each timestep repeats this in the yaml.
                if 'ul: {x:' in line and ulx is None:
                    ul = line
                    ulx= ul[14:23]
                    uly= ul[27:38]
                    #print(tile_list_updated[j], ulx, uly)    
                    #print(tile_list_updated[j], lrx, lry)
        print(str('gdal_translate -a_srs EPSG:3577 -a_ullr '+ulx+uly+' '+lrx+lry+' Surat_2yr_'+tile_list_updated[j]+'_'+year+'.tif Surat_2yr_'+tile_list_updated[j]+'_'+year+'_cor.tif'))

gdal_translate -a_srs EPSG:3577 -a_ullr 1500000.0 -2600000.0 1600000.0 -2700000.0 Surat_2yr_15_-27_20100101__20120101.tif Surat_2yr_15_-27_20100101__20120101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1500000.0 -2700000.0 1600000.0 -2800000.0 Surat_2yr_15_-28_20100101__20120101.tif Surat_2yr_15_-28_20100101__20120101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1500000.0 -2800000.0 1600000.0 -2900000.0 Surat_2yr_15_-29_20100101__20120101.tif Surat_2yr_15_-29_20100101__20120101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1500000.0 -2900000.0 1600000.0 -3000000.0 Surat_2yr_15_-30_20100101__20120101.tif Surat_2yr_15_-30_20100101__20120101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1500000.0 -3000000.0 1600000.0 -3100000.0 Surat_2yr_15_-31_20100101__20120101.tif Surat_2yr_15_-31_20100101__20120101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1500000.0 -3100000.0 1600000.0 -3200000.0 Surat_2yr_15_-32_20100101__20120101.tif Surat_2yr_15_-32_20100101__20120101_cor.tif
gdal

gdal_translate -a_srs EPSG:3577 -a_ullr 1700000.0 -2700000.0 1800000.0 -2800000.0 Surat_2yr_17_-28_20140101__20160101.tif Surat_2yr_17_-28_20140101__20160101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1700000.0 -2800000.0 1800000.0 -2900000.0 Surat_2yr_17_-29_20140101__20160101.tif Surat_2yr_17_-29_20140101__20160101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1700000.0 -2900000.0 1800000.0 -3000000.0 Surat_2yr_17_-30_20140101__20160101.tif Surat_2yr_17_-30_20140101__20160101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1700000.0 -3000000.0 1800000.0 -3100000.0 Surat_2yr_17_-31_20140101__20160101.tif Surat_2yr_17_-31_20140101__20160101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1700000.0 -3100000.0 1800000.0 -3200000.0 Surat_2yr_17_-32_20140101__20160101.tif Surat_2yr_17_-32_20140101__20160101_cor.tif
gdal_translate -a_srs EPSG:3577 -a_ullr 1700000.0 -3200000.0 1800000.0 -3300000.0 Surat_2yr_17_-33_20140101__20160101.tif Surat_2yr_17_-33_20140101__20160101_cor.tif
gdal

In [14]:
print (subprocess)

<module 'subprocess' from '/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/subprocess.py'>


In [52]:
#subprocess.run(["ls", "-l"])
result = subprocess.run(["gdal_translate", " -a_srs EPSG:3577"," -a_ullr 1600000.0 -2900000.0 1700000.0 -3000000.0", " Surat_2yr_16_-30_20120101__20140101.tif", " Surat_2yr_16_-30_20120101__20140101_cor.tif"], stderr=True)

<module 'subprocess' from '/g/data/v10/public/modules/dea-env/20180629/lib/python3.6/subprocess.py'>


In [32]:
os.system('gdal_translate -a_srs EPSG:3577 -a_ullr 1800000.0 -3100000.0 1900000.0 -3200000.0 Surat_2yr_18_-32_20120101__20140101.tif Surat_2yr_18_-32_20120101__20140101_cor.tif')

0

In [ ]:
with open(spatial_files[0], 'r') as spatialfileopen:
    #print(spatialfileopen.readlines())
    ulx=uly=lrx=lry=None
    for line in spatialfileopen:
        #if not uly and uly
        if 'ul: {x:' in line and ulx is None:
            ul = line
            ulx= ul[14:23]
            uly = ul[27:38]
            print(line, ulx, uly)

In [16]:
!pwd
#open files. make dictionary of upper left and lower right coords
#if file name includes tile, 

/g/data1a/r78/datacube_stats/TCW_stats/Surat/Surat_2yr/outputs_corrected
